In [103]:
import pandas as pd
import numpy as np
import datetime

### 주가변동에 따른 뉴스기사 labeling

In [88]:
# 데이터 불러오기, 전처리
kospi = 'CJ제일제당'
stock_df = pd.read_csv('../../../Stock_Data/Data/stock_df/stock_df/{}_stock.csv'.format(kospi))
change_df = stock_df[['Date','Close']]

article_df = pd.read_csv('../../../Crawling/Data/kospi20/{}_df.csv'.format(kospi))
article_df = article_df.drop_duplicates('Title',keep = 'first')
article_df = article_df.drop(article_df[article_df['Date'] == '2019.12.31'].index)
article_df = article_df.dropna(axis=0)
article_df = article_df.reset_index(drop=True)
for i in range(len(article_df)):
    article_df['Date'][i] = article_df['Date'][i].replace('.','-')
del article_df['Unnamed: 0']


C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [104]:
# 워킹데이 리스트 불러오기
date_df = pd.read_csv('../../../Stock_Data/Data/stock_df/date_list.csv')
del date_df['Unnamed: 0']
del date_df['article']
date_df = date_df[date_df['Date'] <= '2019-12-31']

while True:
    index = article_df['Date'].isin(date_df['Date']) == False
    holidays = article_df['Date'][index]
    #print(len(holidays))
    if len(holidays) == 0:
        break
    else:
        for i in range(len(holidays)):
            holiday = holidays.iloc[i]
            conv_hd = datetime.datetime.strptime(holiday,'%Y-%m-%d')
            not_holiday = str(conv_hd + datetime.timedelta(days=1))[:10]
            holidays.iloc[i] = not_holiday
        article_df['Date'][index] = holidays 

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [112]:
article_df

,Title,Date,Category
0,"에너지공단, '2019년 ESP 성과공유 워크숍' 개최",2019-11-22,2.0
1,"[공직자 재산공개] 조국 민정수석, '1800만원 VS 1억원' 아까운 선택(종합)",2018-03-29,2.0
2,"[공직자 재산공개] 조국 민정수석, '1800만원 VS 1억원' 아까운 선택",2018-03-29,2.0
3,"CJ제일제당, 임산부 직원 배려하는 조직문화 강화",2018-03-12,2.0
4,'이건희 동영상' 촬영·협박 CJ 前부장 1심서 징역 4년6개월,2017-08-25,2.0
...,...,...,...
2413,"CJ家 장남 이선호 씨, 이다희 전 아나운서와 비공개 결혼…지인 소개로 만나",2018-10-08,11.0
2414,"'윰블리' 정유미, 광고계 블루칩 등극, 윤식당 효과 '톡톡'",2017-05-24,11.0
2415,"[동계 AG] 권이준, 스노보드 남자 하프파이프 은메달",2017-02-27,11.0
2416,"CJ 채용, 18일 오후 6시 마감…지원 자격과 채용 분야는?",2014-03-17,11.0


In [89]:
# 주가 변동률 컬럼 생성

rocs = []
for i in range(len(change_df)):
    if i != 0:
        yesterday = change_df['Close'][(i-1)]
        today = change_df['Close'][i]
        roc = (today-yesterday)/yesterday
        rocs.append(roc)
    else:
        rocs.append(0)
change_df['RateofChange'] = rocs

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [96]:
# 변동률 < 0: -1, 변동률 > 0: 1로 인코딩
change_df['Label'] = np.zeros(len(change_df),)
change_df['Label'].loc[change_df['RateofChange'] < 0] = -1
change_df['Label'].loc[change_df['RateofChange'] > 0] = 1

# 첫날(2014/01/02)는 전날 데이터가 없으므로 다음날과 동일하게 설정
change_df['Label'][0] = change_df['Label'][1]

# 변동률이 0일 경우, 전날 변동 추이 반영
for i in range(len(change_df)):
    if change_df['Label'][i]==0:
        change_df['Label'][i] = change_df['Label'][(i-1)]

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [97]:
np.unique(change_df.Label)

array([-1.,  1.])

In [116]:
merge_df = pd.merge(change_df, article_df, left_on=['Date'], right_on=['Date'], how='right')

In [120]:
df_cj = merge_df[['Title','Label']]
df_cj.to_csv('../../Data/PriceChange/CJ_PC_traindata.csv')

In [115]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [119]:
len(df_cj)

2419